<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C2/W3/ungraded_lab/C2_W3_Lab_1_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Transfer Learning

In this lab, you will see how you can use a pre-trained model to achieve good results even with a small training dataset. This is called _transfer learning_ and you do this by leveraging the trained layers of an existing model and adding your own layers to fit your application. For example, you can:

1. just get the convolution layers of one model
2. attach some dense layers onto it
3. train just the dense network
4. evaluate the results

Doing this will allow you to save time building your application because you will essentially skip weeks of training time of very deep networks. You will just use the features it has learned and tweak it for your dataset. Let's see how these are done in the next sections.

**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Running the notebook on your local machine might result in some of the code blocks throwing errors.

## Setup the pretrained model

You will need to prepare pretrained model and configure the layers that you need. For this exercise, you will use the convolution layers of the [InceptionV3](https://arxiv.org/abs/1512.00567) architecture as your base model. To do that, you need to:

1. Set the input shape to fit your application. In this case. set it to `150x150x3` as you've been doing in the last few labs.

2. Pick and freeze the convolution layers to take advantage of the features it has learned already.

3. Add dense layers which you will train.

Let's see how to do these in the next cells.

First, in preparing the input to the model, you want to fetch the pretrained weights of the `InceptionV3` model and remove the fully connected layer at the end because you will be replacing it later. You will also specify the input shape that your model will accept. Lastly, you want to freeze the weights of these layers because they have been trained already.

In [1]:
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2022-10-29 19:14:20--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
storage.googleapis.com をDNSに問いあわせています... 172.217.175.16, 172.217.175.48, 172.217.175.80, ...
storage.googleapis.com|172.217.175.16|:443 に接続しています... 接続しました。
警告: storage.googleapis.com の証明書(発行者: /C=US/O=Google Trust Services LLC/CN=GTS CA 1C3)の検証に失敗しました:
  発行者の権限を検証できませんでした。
警告: 証明書に記載されている名前 `*.storage.googleapis.com' とホスト名 `storage.googleapis.com' が一致しません
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 87910968 (84M) [application/x-hdf]
`/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5' に保存中

     0K .......... .......... .......... .......... ..........  0% 2.10M 40s
    50K .......... .......... .......... .......... ..........  0% 2.59M 36s
   100K .......... .......... .......... .......... ..........  0% 4.61M 30s
   150K .......... .......... .....

In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

# Set the weights file you downloaded into a variable
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize the base model.
# Set the input shape and remove the dense layers.
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

# Load the pre-trained weights you downloaded.
pre_trained_model.load_weights(local_weights_file)

# Freeze the weights of the layers.
for layer in pre_trained_model.layers:
  layer.trainable = False

You can see the summary of the model below. You can see that it is a very deep network. You can then select up to which point of the network you want to use. As Laurence showed in the exercise, you will use up to `mixed_7` as your base model and add to that. This is because the original last layer might be too specialized in what it has learned so it might not translate well into your application. `mixed_7` on the other hand will be more generalized and you can start with that for your application. After the exercise, feel free to modify and use other layers to see what the results you get.

In [3]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [4]:
# Choose `mixed_7` as the last layer of your base model
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


## Add dense layers for your classifier

Next, you will add dense layers to your model. These will be the layers that you will train and is tasked with recognizing cats and dogs. You will add a [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layer as well to regularize the output and avoid overfitting.

In [5]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

# Append the dense network to the base model
model = Model(pre_trained_model.input, x) 

# Print the model summary. See your dense network connected at the end.
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [6]:
# Set the training parameters
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

## Prepare the dataset

Now you will prepare the dataset. This is basically the same code as the one you used in the data augmentation lab.

In [8]:
# Download the dataset
!wget --no-check-certificate https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2022-10-29 19:15:45--  https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip
storage.googleapis.com をDNSに問いあわせています... 216.58.220.112, 142.250.196.112, 142.250.196.144, ...
storage.googleapis.com|216.58.220.112|:443 に接続しています... 接続しました。
警告: storage.googleapis.com の証明書(発行者: /C=US/O=Google Trust Services LLC/CN=GTS CA 1C3)の検証に失敗しました:
  発行者の権限を検証できませんでした。
警告: 証明書に記載されている名前 `*.storage.googleapis.com' とホスト名 `storage.googleapis.com' が一致しません
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 68606236 (65M) [application/zip]
`cats_and_dogs_filtered.zip' に保存中

     0K .......... .......... .......... .......... ..........  0% 1.48M 44s
    50K .......... .......... .......... .......... ..........  0% 2.39M 36s
   100K .......... .......... .......... .......... ..........  0% 5.34M 28s
   150K .......... .......... .......... .......... ..........  0% 4.80M 24s
   200K ..

In [9]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Extract the archive
zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

# Define our example directories and files
base_dir = 'tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')

# Directory with training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats') 

# Directory with training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') 

# Directory with validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') 

# Directory with validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Train the model

With that, you can now train the model. You will do 20 epochs and plot the results afterwards.

In [10]:
# Train the model.
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)

Epoch 1/20
100/100 - 21s - loss: 0.3689 - accuracy: 0.8560 - val_loss: 0.1018 - val_accuracy: 0.9660 - 21s/epoch - 209ms/step
Epoch 2/20
100/100 - 12s - loss: 0.2281 - accuracy: 0.9050 - val_loss: 0.1226 - val_accuracy: 0.9570 - 12s/epoch - 117ms/step
Epoch 3/20
100/100 - 11s - loss: 0.1852 - accuracy: 0.9295 - val_loss: 0.1328 - val_accuracy: 0.9630 - 11s/epoch - 113ms/step
Epoch 4/20
100/100 - 12s - loss: 0.1933 - accuracy: 0.9350 - val_loss: 0.1445 - val_accuracy: 0.9610 - 12s/epoch - 122ms/step
Epoch 5/20
100/100 - 12s - loss: 0.1751 - accuracy: 0.9390 - val_loss: 0.1330 - val_accuracy: 0.9610 - 12s/epoch - 122ms/step
Epoch 6/20
100/100 - 13s - loss: 0.1895 - accuracy: 0.9390 - val_loss: 0.1217 - val_accuracy: 0.9670 - 13s/epoch - 131ms/step
Epoch 7/20
100/100 - 13s - loss: 0.1678 - accuracy: 0.9465 - val_loss: 0.1301 - val_accuracy: 0.9660 - 13s/epoch - 131ms/step
Epoch 8/20
100/100 - 13s - loss: 0.1678 - accuracy: 0.9490 - val_loss: 0.1592 - val_accuracy: 0.9560 - 13s/epoch - 134

## Evaluate the results

You will use the same code to plot the results. As you can see, the validation accuracy is also trending upwards as your training accuracy improves. This is a good sign that your model is no longer overfitting!

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()